In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
imgs_medseg=np.load("../input/covid-segmentation/images_medseg.npy")
msks_medseg=np.load("../input/covid-segmentation/masks_medseg.npy")

In [ ]:
import matplotlib.pyplot as plt
def normalize_img(img):
    img = img.astype("float32")
    img[img > 500] = 500
    img[img < -1500] = -1500
    min_perc, max_perc = np.percentile(img, 5), np.percentile(img, 95)
    img_valid = img[(img > min_perc) & (img < max_perc)]
    mean, std = img_valid.mean(), img_valid.std()
    img = (img-mean)/std
    return img
def plot_hists(images):
    plt.hist(images.ravel(), bins=100, density=True, color='b', alpha=1)
print(imgs_medseg.shape)
imgs_medseg = normalize_img(imgs_medseg)
plot_hists(imgs_medseg)

In [ ]:
from sklearn.model_selection import train_test_split
train_x, val_x, train_y, val_y = train_test_split(imgs_medseg, msks_medseg, test_size=0.1, random_state=42)
print(train_x[:10].shape)
print(msks_medseg[0])

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
def convolution(input, num_filters):
  x = Conv2D(num_filters, 3, padding="same")(input)
  x = BatchNormalization()(x)
  x = Activation("relu")(x)

  x = Conv2D(num_filters, 3, padding="same")(x)
  x = BatchNormalization()(x)
  x = Activation("relu")(x)
  return x

def downsample(input, num_filters):
  x = convolution(input, num_filters)
  p = MaxPool2D((2,2))(x)
  return x, p

def upsample(input, skip_connections, num_filters):
  x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
  x = Concatenate()([x, skip_connections])
  x = convolution(x, num_filters)
  return x

def build_unet(input_shape):
  inputs = Input(input_shape)
  
  sc1, p1 = downsample(inputs, 64)
  sc2, p2 = downsample(p1, 128)
  sc3, p3 = downsample(p2, 256)
  sc4, p4 = downsample(p3, 512)

  b1 = convolution(p4, 1024)

  d1 = upsample(b1, sc4, 512)
  d2 = upsample(d1, sc3, 256)
  d3 = upsample(d2, sc2, 128)
  d4 = upsample(d3, sc1, 64)

  outputs = Conv2D(4,(1,1),padding="same",activation="softmax")(d4)

  model = Model(inputs, outputs, name="U-Net")
  return model


In [ ]:
checkpoint_path = "./cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
unet = build_unet(imgs_medseg.shape[1:])
unet.compile(loss="binary_crossentropy", optimizer="adam", metrics=["acc"])
results = unet.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=100, batch_size=1, verbose=1)


In [ ]:
from keras.models import model_from_json
model_json = unet.to_json()
with open("./model.json", "w") as json_file:
    json_file.write(model_json)
unet.save_weights("./model.h5")
print("Saved model to disk")



In [ ]:
import cv2
a=cv2.imread("../input/ct-sample/pCT100.jpg")
gray = cv2.cvtColor(a, cv2.COLOR_BGR2GRAY)
gray=np.expand_dims(gray, axis=2)
gray=np.expand_dims(gray, axis=0)
gray=normalize_img(gray)
k=unet.predict(gray)
print(k.shape)
%matplotlib inline
plot_hists(k[0,:,:,3])
# plot_hists(k)
import numpy.ma as ma
# k=np.reshape(k,(1,512,512,4))

print(imgs_medseg[0])
import matplotlib.pyplot as plt
plot_hists(test)
print(plt.imshow(ma.masked_array(gray,k[:,:,:,3]).reshape(512,512)))